# Tóm tắt tin tức từ all_news.csv

Notebook này sử dụng **Google Gemini AI** và **TOON format** để:
- 📰 Tóm tắt tin tức tài chính từ `all_news.csv`
- 🎯 Tự động nhận diện mã cổ phiếu
- 💰 **Tiết kiệm 30-60% token** nhờ TOON format
- ⚡ Tăng tốc độ phản hồi và giảm chi phí API

---

## 🎯 Điểm nổi bật

✨ **TOON Format Integration** - Định dạng tối ưu cho LLM, giảm token hiệu quả  
🔄 **Retry Logic** - Tự động thử lại khi gặp lỗi  
💾 **Checkpoint System** - Lưu tiến độ, tiếp tục được khi bị gián đoạn  
🤖 **Multi-model Strategy** - Tự động chuyển model khi cần  
📊 **Stock Code Detection** - Nhận diện thông minh mã cổ phiếu từ nội dung

In [9]:
# Import các thư viện cần thiết
import pandas as pd
import os
import time
import re
import json
import subprocess
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
import google.generativeai as genai

# Load environment variables
load_dotenv()

class APIKeyManager:
    """
    Quản lý nhiều Google API keys và tự động chuyển đổi khi gặp lỗi.
    
    Quy tắc:
    - Mỗi key được thử tối đa 2 lần
    - Sau 2 lần lỗi → tự động chuyển key tiếp theo
    - Hết key → báo lỗi
    """
    
    MAX_RETRIES_PER_KEY = 2  # Số lần thử tối đa cho mỗi key
    
    def __init__(self):
        """Khởi tạo và load tất cả API keys từ .env"""
        # Load các API keys từ environment
        self.keys = [
            ("GOOGLE_API_KEY", os.getenv("GOOGLE_API_KEY")),
            ("GOOGLE_API_KEY_2", os.getenv("GOOGLE_API_KEY_2")),
            ("GOOGLE_API_KEY_3", os.getenv("GOOGLE_API_KEY_3")),
            ("GOOGLE_API_KEY_4", os.getenv("GOOGLE_API_KEY_4")),
        ]
        
        # Chỉ giữ lại các key hợp lệ (không None)
        self.keys = [(name, key) for name, key in self.keys if key]
        
        if not self.keys:
            raise ValueError("❌ Không tìm thấy API key! Kiểm tra file .env")
        
        # Khởi tạo trạng thái
        self.current_index = 0
        self.error_counts = {name: 0 for name, _ in self.keys}  # Đếm lỗi mỗi key
        
        print(f"✓ Phát hiện {len(self.keys)} API keys")
        self._activate_key(0)
    
    def _activate_key(self, index):
        """Kích hoạt API key tại vị trí index"""
        if index >= len(self.keys):
            raise Exception("❌ Đã hết tất cả API keys!")
        
        self.current_index = index
        key_name, key_value = self.keys[index]
        
        # Cấu hình Google AI với key mới
        genai.configure(api_key=key_value)
        
        print(f"🔑 Đang sử dụng: {key_name} (Key {index + 1}/{len(self.keys)})")
    
    def get_current_key(self):
        """Lấy API key hiện tại"""
        return self.keys[self.current_index][1]
    
    def get_models(self):
        """
        Tạo các model AI với API key hiện tại.
        
        Returns:
            tuple: (model, model_more_temp, model_pro)
        """
        current_key = self.get_current_key()
        
        model = ChatGoogleGenerativeAI(
            model="gemini-2.0-flash-lite", 
            temperature=0.02,
            google_api_key=current_key
        )
        
        model_more_temp = ChatGoogleGenerativeAI(
            model="gemini-2.0-flash-lite", 
            temperature=0.1,
            google_api_key=current_key
        )
        
        model_pro = ChatGoogleGenerativeAI(
            model="gemini-2.5-pro-exp-03-25", 
            temperature=0.1,
            google_api_key=current_key
        )
        
        return model, model_more_temp, model_pro
    
    def on_error(self):
        """
        Xử lý khi gặp lỗi API.
        
        Khi hết tất cả keys → Reset và quay lại key đầu tiên
        
        Returns:
            bool: Luôn trả về True (vì sẽ tự động reset và tiếp tục)
        """
        key_name, _ = self.keys[self.current_index]
        self.error_counts[key_name] += 1
        
        error_count = self.error_counts[key_name]
        print(f"⚠ Lỗi lần {error_count} với {key_name}")
        
        # Nếu đã đạt giới hạn retry cho key này
        if error_count >= self.MAX_RETRIES_PER_KEY:
            print(f"⛔ {key_name} đã lỗi {error_count}/{self.MAX_RETRIES_PER_KEY} lần")
            
            # Thử chuyển sang key tiếp theo
            next_index = self.current_index + 1
            
            if next_index < len(self.keys):
                print(f"🔄 Chuyển sang key tiếp theo...")
                self._activate_key(next_index)
                return True  # Đã chuyển key thành công
            else:
                # Hết tất cả keys → Reset và quay lại đầu
                print(f"🔄 Đã thử hết {len(self.keys)} keys! Reset và quay lại key đầu tiên...")
                
                # Reset tất cả error counts
                for key, _ in self.keys:
                    self.error_counts[key] = 0
                
                # Quay lại key đầu tiên
                self._activate_key(0)
                print("✓ Đã reset tất cả keys và bắt đầu lại từ đầu")
                return True  # Tiếp tục với key đầu tiên
        
        # Vẫn còn retry cho key hiện tại
        return True
    
    def reset_error_count(self):
        """Reset đếm lỗi của key hiện tại (sau khi thành công)"""
        key_name, _ = self.keys[self.current_index]
        self.error_counts[key_name] = 0

api_manager = APIKeyManager()

print("✓ Đã khởi tạo APIKeyManager với 4 API keys")

✓ Phát hiện 4 API keys
🔑 Đang sử dụng: GOOGLE_API_KEY (Key 1/4)
✓ Đã khởi tạo APIKeyManager với 4 API keys


In [10]:
# Test: Đọc và kiểm tra dữ liệu
def test_read_news():
    """
    Kiểm tra xem có đọc được file all_news.csv không
    """
    try:
        df_test = pd.read_csv("all_news/all_news.csv", nrows=3)
        print(f"✓ Đọc thành công file all_news.csv")
        print(f"\n📊 Các cột: {df_test.columns.tolist()}")
        print(f"\n📝 Mẫu dữ liệu (3 dòng đầu):")
        print(df_test[['postID', 'date', 'taggedSymbols']].head())
        
        # Kiểm tra combinedContent
        if len(df_test) > 0:
            print(f"\n📄 Mẫu combinedContent:")
            print(df_test.iloc[0]['combinedContent'][:200] + "...")
        
        return True
    except Exception as e:
        print(f"❌ Lỗi khi đọc file: {e}")
        return False

# Chạy test (bỏ comment để kiểm tra)
# test_read_news()

In [11]:
# Định nghĩa prompt template tối ưu với TOON format
news_summarize_with_stock_template = PromptTemplate.from_template("""
Bạn là một chuyên gia phân tích thị trường chứng khoán Việt Nam. 

Dữ liệu đầu vào có thể ở định dạng TOON (Token-Oriented Object Notation) - một format tiết kiệm token.
Nếu thấy ```toon block, dữ liệu sẽ có dạng bảng với các trường: id, taggedSymbols, title, content.

Nhiệm vụ của bạn:
1. Đọc dữ liệu từ format TOON hoặc text thông thường
2. Tóm tắt ngắn gọn nhưng ĐẦY ĐỦ thông tin quan trọng:
   - BẮT BUỘC giữ nguyên TẤT CẢ số liệu (%, số tiền, số lượng, tỷ lệ...)
   - GHI RÕ chi tiết, lý do, nguyên nhân
   - Câu văn hoàn chỉnh, rõ chủ ngữ - vị ngữ
   - Độ dài phù hợp với tầm quan trọng của tin
3. Xác định CHÍNH XÁC mã cổ phiếu liên quan (ưu tiên cao nhất)

Quy tắc nhận diện mã cổ phiếu:
1. Ưu tiên sử dụng mã từ taggedSymbols nếu có và chính xác
2. Nếu taggedSymbols trống/sai, TỰ NHẬN DIỆN từ nội dung:
   - Tìm TÊN CÔNG TY (VD: FPT, Vietcombank, Vingroup, Hòa Phát...)
   - Chuyển thành MÃ CHỨNG KHOÁN chính xác (FPT→FPT, Vietcombank→VCB, Vingroup→VIC, Hòa Phát→HPG)
   - Nhiều công ty → liệt kê TẤT CẢ mã, cách nhau bởi dấu phẩy
3. Chỉ nói ngành/lĩnh vực chung → để trống mã
4. MÃ phải viết HOA và chính xác theo HOSE/HNX/UPCOM

Danh sách công ty phổ biến:
FPT→FPT | Vietcombank→VCB | Vingroup/Vinhomes→VIC,VHM | Hòa Phát→HPG | PV Gas→GAS
Masan→MSN,MCH | Mobile World→MWG | SSI→SSI | Viettel→VTP,CTR,VGI | Sabeco→SAB
Vinamilk→VNM | Techcombank→TCB | BIDV→BID | VietinBank→CTG | VPBank→VPB

Dữ liệu đầu vào:
{articles_list}

Format đầu ra (MỖI TIN MỘT DÒNG):
[MÃ hoặc trống]: [Tiêu đề ngắn] | [Tóm tắt chi tiết với số liệu]

Ví dụ:
FPT: FPT mua lại 5 triệu cổ phiếu quỹ | FPT Corporation mua tối đa 5 triệu cp quỹ Q1/2025, giá ~120K đồng/cp, tổng 600 tỷ đồng, tối ưu vốn.
VCB: Vietcombank lãi Q4 tăng 18% | VCB lợi nhuận Q4/2024 đạt 9.200 tỷ (+18% YoY), tín dụng +15%, dịch vụ +22%.
HPG, VHM: Thép và BĐS hưởng lợi gói 120K tỷ | Gói kích cầu 120K tỷ tập trung hạ tầng, nhà XH; HPG nhu cầu thép +12%, VHM dự án +30%.
: VN-Index tăng nhờ dòng tiền ngoại | VN-Index +8.5 điểm lên 1.245, thanh khoản 18.5K tỷ, ngoại mua ròng 450 tỷ.

Lưu ý:
- Không dùng ":" trong nội dung tóm tắt
- GHI RÕ số liệu (%, số tiền, số lượng...)
- Nhiều mã → ghi hết, cách bằng dấu phẩy
- Không xác định được mã → để trống
- Tổng hợp 20 tin quan trọng nhất trong các tin đã cho
- Ưu tiên tin ảnh hưởng lớn
- Đầu ra phải là các câu văn hoàn chỉnh, rõ ràng được tóm tắt từ các tin gốc đã cho


Tóm tắt:
""")

In [12]:
def create_chains(api_manager):
    """
    Tạo hoặc cập nhật các chains với models từ APIKeyManager
    
    Hàm này nên được gọi:
    - Lần đầu tiên khởi tạo
    - Mỗi khi api_manager chuyển sang API key mới
    
    Parameters:
    -----------
    api_manager : APIKeyManager
        Manager chứa các models đã được cấu hình
        
    Returns:
    --------
    Tuple[Chain, Chain, Chain] : (chain_summary, chain_summary_more_temp, chain_summary_pro)
    """
    # Lấy models từ api_manager (đã được khởi tạo với API key hiện tại)
    model, model_more_temperature, model_pro = api_manager.get_models()
    
    # Tạo chains
    chain_summary = news_summarize_with_stock_template | model
    chain_summary_more_temp = news_summarize_with_stock_template | model_more_temperature
    chain_summary_pro = news_summarize_with_stock_template | model_pro
    
    return chain_summary, chain_summary_more_temp, chain_summary_pro

# Khởi tạo lần đầu với api_manager
chain_summary, chain_summary_more_temp, chain_summary_pro = create_chains(api_manager)

print("✓ Đã khởi tạo các chains thành công")

✓ Đã khởi tạo các chains thành công


In [13]:
# Cài đặt TOON format để tiết kiệm token
try:
    import importlib.util
    if importlib.util.find_spec("toon_format") is None:
        print("📦 Đang cài đặt TOON format...")
        subprocess.check_call(["pip", "install", "toon-format"], stdout=subprocess.DEVNULL)
        print("✓ Đã cài đặt TOON format")
    else:
        print("✓ TOON format đã được cài đặt")
except Exception as e:
    print(f"⚠ Không thể cài đặt TOON: {e}")
    print("   Sẽ sử dụng format JSON thông thường")

✓ TOON format đã được cài đặt


In [14]:
BASE_DELAY = 30  # Delay cơ bản giữa các lần retry (giây)

def invoke_chain_with_retry(chain, prompt, api_manager, base_delay=BASE_DELAY):
    """
    Gọi chain với cơ chế retry tự động và tích hợp APIKeyManager
    
    Hệ thống sẽ tự động:
    - Thử lại với cùng API key (tối đa 2 lần theo MAX_RETRIES_PER_KEY)
    - Chuyển sang API key khác khi key hiện tại thất bại
    - Thử tất cả 4 keys trước khi báo lỗi hoàn toàn
    
    Parameters:
    -----------
    chain : Chain
        LangChain chain để invoke
    prompt : dict
        Input prompt cho chain
    api_manager : APIKeyManager
        Manager để quản lý API keys
    base_delay : int
        Delay giữa các lần retry (giây)
        
    Returns:
    --------
    Response object hoặc None nếu tất cả keys đều thất bại
    """
    total_attempts = 0
    max_total_attempts = len(api_manager.keys) * api_manager.MAX_RETRIES_PER_KEY
    
    while total_attempts < max_total_attempts:
        try:
            # Thử gọi API
            response = chain.invoke(prompt)
            
            # Thành công -> reset error count cho key hiện tại
            api_manager.reset_error_count()
            return response
            
        except Exception as e:
            total_attempts += 1
            error_msg = str(e)
            
            # Báo lỗi cho API manager
            switched = api_manager.on_error()
            
            if total_attempts >= max_total_attempts:
                print(f"❌ Đã thử tất cả {len(api_manager.keys)} API keys ({total_attempts} lần) nhưng vẫn lỗi")
                print(f"   Lỗi cuối: {error_msg}")
                return None
            
            # Chờ trước khi retry
            if switched:
                delay = base_delay  # Delay ngắn khi chuyển key mới
                print(f"⏳ Đợi {delay}s trước khi thử key mới...")
            else:
                # Exponential backoff cho cùng một key
                retry_num = api_manager.error_counts.get(api_manager.current_index, 0)
                delay = base_delay * (1.5 ** (retry_num - 1))
                print(f"⏳ Đợi {delay:.0f}s trước khi retry ({retry_num}/{api_manager.MAX_RETRIES_PER_KEY})...")
            
            time.sleep(delay)

def to_toon_format(data):
    """
    Chuyển đổi dữ liệu sang TOON format để tiết kiệm token
    TOON giúp giảm 30-60% token so với JSON cho dữ liệu dạng bảng
    """
    try:
        # Thử import và sử dụng TOON
        import subprocess
        import tempfile
        
        # Chuyển data sang JSON
        json_data = json.dumps(data, ensure_ascii=False)
        
        # Sử dụng npx để chuyển đổi sang TOON
        result = subprocess.run(
            ['npx', '-y', '@toon-format/cli', '--delimiter', '\t'],
            input=json_data,
            capture_output=True,
            text=True,
            encoding='utf-8'
        )
        
        if result.returncode == 0 and result.stdout:
            return result.stdout.strip()
        else:
            # Fallback: format đơn giản tương tự TOON
            return json.dumps(data, ensure_ascii=False, indent=2)
    except Exception as e:
        # Nếu lỗi, dùng JSON thông thường
        return json.dumps(data, ensure_ascii=False, indent=2)

def combine_articles(df: pd.DataFrame, use_toon=True) -> str:
    """
    Kết hợp các bài báo thành chuỗi văn bản tối ưu cho LLM
    Sử dụng TOON format để giảm token consumption
    Cấu trúc all_news.csv: postID, date, sentiment, taggedSymbols, combinedContent
    """
    if use_toon and len(df) > 0:
        try:
            # Chuẩn bị dữ liệu dạng bảng cho TOON
            articles_data = []
            for idx, row in df.iterrows():
                # Lấy combinedContent và tách title từ đó
                combined = str(row.get('combinedContent', ''))
                
                # Tách title từ combinedContent (format: "Title: ... Description: ...")
                title = ""
                content = combined[:700]  # Giới hạn độ dài
                
                if "Title:" in combined:
                    parts = combined.split("Description:", 1)
                    title = parts[0].replace("Title:", "").strip()
                    if len(parts) > 1:
                        content = parts[1].strip()[:500]
                
                article = {
                    "id": int(row.get('postID', idx + 1)),
                    "taggedSymbols": str(row.get('taggedSymbols', '[]')),
                    "title": title if title else content[:100],  # Dùng 100 ký tự đầu nếu không có title
                    "content": content
                }
                articles_data.append(article)
            
            # Chuyển sang TOON format
            toon_data = to_toon_format({"articles": articles_data})
            
            # Thêm header giải thích
            return f"```toon\n{toon_data}\n```\n\nLưu ý: Dữ liệu dùng TOON format (tiết kiệm token). Mỗi dòng: id, taggedSymbols, title, content."
            
        except Exception as e:
            print(f"⚠ Không thể dùng TOON, dùng format thông thường: {e}")
    
    # Fallback: format text thông thường
    articles = []
    for idx, row in df.iterrows():
        tagged_symbols = row.get('taggedSymbols', '[]')
        combined = str(row.get('combinedContent', ''))
        
        article_text = f"Tin {idx+1} (postID: {row.get('postID', 'N/A')}):\n"
        article_text += f"Tagged Symbols: {tagged_symbols}\n"
        article_text += f"Nội dung: {combined[:500]}...\n"  # Giới hạn độ dài
        articles.append(article_text)
    
    return "\n---\n".join(articles)

def parse_summary_response(response, date, starting_index=1):
    """
    Phân tích kết quả tóm tắt từ LLM theo định dạng [MÃ]: [Tiêu đề] | [Nội dung]
    """
    pattern = r'^([A-Z, ]*?):\s*(.+?)\s*\|\s*(.+)$'
    response_text = str(response.content)
    lines = response_text.splitlines()
    articles = []

    for line in lines:
        line = line.strip()
        if not line:
            continue
        match = re.match(pattern, line)
        if match:
            stock_codes, title, description = match.groups()
            
            # Xử lý mã cổ phiếu: loại bỏ khoảng trắng thừa
            stock_codes = stock_codes.strip()
            
            articles.append({
                "postID": starting_index,
                "stockCodes": stock_codes if stock_codes else "",
                "title": title.strip(),
                "description": description.strip(),
                "date": date
            })
            starting_index += 1
    
    return pd.DataFrame(articles)

print("✓ Đã định nghĩa các hàm tiện ích với TOON support")

✓ Đã định nghĩa các hàm tiện ích với TOON support


In [15]:
# Hàm chính để tóm tắt tin tức với TOON format support
def summarize_news_with_stock_codes(
    csv_path="all_news/all_news.csv",
    output_path="summarized_news_with_stocks.csv",
    batch_size=5,
    start_date=None,
    end_date=None,
    max_articles_per_day=500,
    use_toon=True  # Bật TOON format để tiết kiệm token
):
    """
    Tóm tắt tin tức từ all_news.csv với tự động nhận diện mã cổ phiếu
    
    Parameters:
    -----------
    csv_path : str
        Đường dẫn đến file CSV chứa tin tức gốc
    output_path : str
        Đường dẫn file CSV kết quả
    batch_size : int
        Số ngày xử lý mỗi batch (để lưu checkpoint)
    start_date : str, optional
        Ngày bắt đầu (format: YYYY-MM-DD), nếu None sẽ xử lý từ đầu
    end_date : str, optional
        Ngày kết thúc (format: YYYY-MM-DD), nếu None sẽ xử lý đến cuối
    max_articles_per_day : int
        Số lượng bài báo tối đa mỗi ngày (để tránh vượt quá context length)
    use_toon : bool
        Sử dụng TOON format để tiết kiệm 30-60% token (mặc định: True)
    """
    print(f"📖 Đang đọc dữ liệu từ {csv_path}...")
    
    # Đọc dữ liệu
    df = pd.read_csv(csv_path)
    print(f"✓ Đã đọc {len(df)} bài báo")
    
    # Chuyển đổi cột date sang datetime (cột đã tên là 'date' sẵn)
    df['parsed_date'] = pd.to_datetime(df['date'])
    df['only_date'] = df['parsed_date'].dt.date
    
    # Lọc theo khoảng thời gian nếu được chỉ định
    if start_date:
        start_dt = pd.to_datetime(start_date).date()
        df = df[df['only_date'] >= start_dt]
        print(f"✓ Lọc từ ngày {start_date}: còn {len(df)} bài")
    
    if end_date:
        end_dt = pd.to_datetime(end_date).date()
        df = df[df['only_date'] <= end_dt]
        print(f"✓ Lọc đến ngày {end_date}: còn {len(df)} bài")
    
    # Sắp xếp theo thời gian tăng dần
    df = df.sort_values('date')
    
    # Khởi tạo DataFrame kết quả
    df_summarized = pd.DataFrame(columns=["postID", "stockCodes", "title", "description", "date"])
    
    # Nhóm theo ngày
    date_groups = df.groupby("only_date")
    total_groups = len(date_groups)
    
    print(f"\n📅 Sẽ xử lý {total_groups} ngày")
    print(f"⚙ Batch size: {batch_size} ngày/batch")
    print(f"⚙ Tối đa {max_articles_per_day} bài/ngày")
    print(f"⚙ TOON format: {'✓ BẬT (tiết kiệm 30-60% token)' if use_toon else '✗ TẮT'}\n")
    
    # Kiểm tra checkpoint hiện có
    processed_dates = set()  # Tập hợp các ngày đã xử lý
    
    if os.path.exists(output_path):
        existing_df = pd.read_csv(output_path)
        if len(existing_df) > 0:
            # Lấy tất cả các ngày đã được xử lý
            existing_df['existing_parsed_date'] = pd.to_datetime(existing_df['date'])
            existing_df['existing_only_date'] = existing_df['existing_parsed_date'].dt.date
            processed_dates = set(existing_df['existing_only_date'].unique())
            
            print(f"🔄 Phát hiện checkpoint: đã có {len(processed_dates)} ngày được xử lý")
            df_summarized = existing_df
            
            # Lọc bỏ các ngày đã xử lý
            date_groups = [(d, g) for d, g in date_groups if d not in processed_dates]
            total_groups = len(date_groups)
            print(f"✓ Còn {total_groups} ngày cần xử lý (đã skip {len(processed_dates)} ngày)\n")
        else:
            print(f"⚠ File checkpoint trống, bắt đầu từ đầu\n")
    else:
        print(f"⚠ Chưa có checkpoint, bắt đầu xử lý mới\n")
    
    # Xử lý theo batch
    current_idx = len(df_summarized) + 1
    
    for batch_idx in range(0, total_groups, batch_size):
        batch_end = min(batch_idx + batch_size, total_groups)
        batch_dates = list(date_groups)[batch_idx:batch_end]
        
        print(f"\n{'='*60}")
        print(f"📦 BATCH {batch_idx//batch_size + 1}: Xử lý ngày {batch_idx+1} đến {batch_end}/{total_groups}")
        print(f"{'='*60}")
        
        batch_summaries = []
        
        for i, (date, group) in enumerate(batch_dates):
            date_position = batch_idx + i + 1
            
            # Giới hạn số lượng bài báo mỗi ngày
            if len(group) > max_articles_per_day:
                print(f"⚠ Ngày {date} có {len(group)} bài, giới hạn xuống {max_articles_per_day} bài")
                group = group.head(max_articles_per_day)
            
            # Kết hợp các bài báo (với TOON nếu được bật)
            combined_articles = combine_articles(group, use_toon=use_toon)
            
            # Tạo prompt
            summary_prompt = {
                "articles_list": combined_articles
            }
            
            print(f"\n[{date_position}/{total_groups}] 📰 Xử lý ngày {date} ({len(group)} bài báo)...")
            
            # Retry với nhiều lần thử
            max_summary_retries = 10
            summary_retry_count = 0
            summary_df = pd.DataFrame()
            
            while summary_retry_count < max_summary_retries:
                # Chọn model dựa trên số lần retry
                if summary_retry_count < 3:
                    selected_chain = chain_summary
                    model_name = "flash-lite (temp=0.02)"
                elif summary_retry_count < 5:
                    selected_chain = chain_summary_more_temp
                    model_name = "flash-lite (temp=0.1)"
                else:
                    selected_chain = chain_summary_pro
                    model_name = "pro (temp=0.1)"
                
                print(f"   🤖 Model: {model_name} | Format: {'TOON' if use_toon else 'Text'}")
                
                # Gọi API với retry logic và API manager
                summary_response = invoke_chain_with_retry(selected_chain, summary_prompt, api_manager)
                time.sleep(1)  # Rate limiting
                
                if summary_response is None:
                    print(f"   ❌ Không thể tóm tắt ngày {date}")
                    break
                
                # Parse kết quả
                date_str = f"{date}T00:00:00+07:00"
                summary_df = parse_summary_response(summary_response, date_str, starting_index=current_idx)
                
                # Kiểm tra kết quả
                if len(summary_df) > 0:
                    print(f"   ✓ Tóm tắt thành {len(summary_df)} tin")
                    break
                
                summary_retry_count += 1
                print(f"   ⚠ Kết quả rỗng, thử lại lần {summary_retry_count}/{max_summary_retries}")
                time.sleep(BASE_DELAY)
            
            # Thêm vào kết quả batch
            if len(summary_df) > 0:
                current_idx += len(summary_df)
                batch_summaries.append(summary_df)
            else:
                print(f"   ❌ Thất bại sau {max_summary_retries} lần thử cho ngày {date}")
        
        # Lưu checkpoint sau mỗi batch
        if batch_summaries:
            batch_df = pd.concat(batch_summaries, ignore_index=True)
            df_summarized = pd.concat([df_summarized, batch_df], ignore_index=True)
            
            # Lưu file
            df_summarized.to_csv(output_path, index=False, encoding='utf-8-sig')
            
            checkpoint_num = batch_idx // batch_size + 1
            print(f"\n💾 Đã lưu checkpoint {checkpoint_num}: {len(df_summarized)} tin tóm tắt")
    
    # Lưu file cuối cùng
    df_summarized.to_csv(output_path, index=False, encoding='utf-8-sig')
    
    print(f"\n{'='*60}")
    print(f"✅ HOÀN THÀNH!")
    print(f"📊 Tổng số tin tóm tắt: {len(df_summarized)}")
    print(f"💾 Đã lưu vào: {output_path}")
    if use_toon:
        print(f"💰 Ước tính tiết kiệm: 30-60% token nhờ TOON format")
    print(f"{'='*60}")
    
    return df_summarized

print("✓ Đã định nghĩa hàm summarize_news_with_stock_codes() với TOON support")

✓ Đã định nghĩa hàm summarize_news_with_stock_codes() với TOON support


In [ ]:
# Chạy tóm tắt tin tức với TOON format
# TOON giúp giảm 30-60% token → Tiết kiệm chi phí API đáng kể!

result_df = summarize_news_with_stock_codes(
    csv_path="all_news/all_news.csv",              # File input
    output_path="summarized_news_with_stocks.csv", # File output
    batch_size=5,                                   # Số ngày/batch
    start_date="2021-09-30",                        # Ngày bắt đầu (None = từ đầu)
    end_date="2022-01-01",                                  # Ngày kết thúc (None = đến cuối)
    max_articles_per_day=500,                        # Tối đa bài/ngày
    use_toon=True                                   # ✓ BẬT TOON để tiết kiệm token
)

📖 Đang đọc dữ liệu từ all_news/all_news.csv...
✓ Đã đọc 241990 bài báo
✓ Lọc từ ngày 2021-09-30: còn 239142 bài
✓ Đã đọc 241990 bài báo
✓ Lọc từ ngày 2021-09-30: còn 239142 bài
✓ Lọc đến ngày 2022-01-01: còn 8816 bài

📅 Sẽ xử lý 94 ngày
⚙ Batch size: 5 ngày/batch
⚙ Tối đa 500 bài/ngày
⚙ TOON format: ✓ BẬT (tiết kiệm 30-60% token)

⚠ Chưa có checkpoint, bắt đầu xử lý mới


📦 BATCH 1: Xử lý ngày 1 đến 5/94

[1/94] 📰 Xử lý ngày 2021-09-30 (109 bài báo)...
   🤖 Model: flash-lite (temp=0.02) | Format: TOON
✓ Lọc đến ngày 2022-01-01: còn 8816 bài

📅 Sẽ xử lý 94 ngày
⚙ Batch size: 5 ngày/batch
⚙ Tối đa 500 bài/ngày
⚙ TOON format: ✓ BẬT (tiết kiệm 30-60% token)

⚠ Chưa có checkpoint, bắt đầu xử lý mới


📦 BATCH 1: Xử lý ngày 1 đến 5/94

[1/94] 📰 Xử lý ngày 2021-09-30 (109 bài báo)...
   🤖 Model: flash-lite (temp=0.02) | Format: TOON
⚠ Lỗi lần 1 với GOOGLE_API_KEY
⏳ Đợi 30s trước khi thử key mới...
⚠ Lỗi lần 1 với GOOGLE_API_KEY
⏳ Đợi 30s trước khi thử key mới...
⚠ Lỗi lần 2 với GOOGLE_API_KEY


## Xem kết quả

Xem một số mẫu tin tức đã tóm tắt:

In [ ]:
# Xem kết quả tóm tắt
print(f"Tổng số tin đã tóm tắt: {len(result_df)}")
print(f"\nCác cột: {result_df.columns.tolist()}")
print(f"\n{'='*80}")
print("MẪU 10 TIN TÓM TẮT ĐẦU TIÊN:")
print(f"{'='*80}\n")

for idx, row in result_df.head(10).iterrows():
    stock_codes = row['stockCodes'] if row['stockCodes'] else "Không có mã"
    print(f"[{row['postID']}] {stock_codes}")
    print(f"📅 {row['date']}")
    print(f"📌 {row['title']}")
    print(f"📝 {row['description']}")
    print(f"{'-'*80}\n")

In [ ]:
# Phân tích thống kê mã cổ phiếu
print("📊 THỐNG KÊ MÃ CỔ PHIẾU ĐƯỢC ĐỀ CẬP\n")

# Đếm số lượng tin có mã và không có mã
has_stock = result_df[result_df['stockCodes'] != ''].shape[0]
no_stock = result_df[result_df['stockCodes'] == ''].shape[0]

print(f"✓ Tin có mã cổ phiếu: {has_stock} ({has_stock/len(result_df)*100:.1f}%)")
print(f"✓ Tin không có mã: {no_stock} ({no_stock/len(result_df)*100:.1f}%)")

# Tách các mã cổ phiếu và đếm tần suất
all_stocks = []
for codes in result_df['stockCodes']:
    if codes:
        # Tách các mã bằng dấu phẩy
        stocks = [s.strip() for s in codes.split(',') if s.strip()]
        all_stocks.extend(stocks)

if all_stocks:
    from collections import Counter
    stock_counter = Counter(all_stocks)
    
    print(f"\n📈 TOP 20 MÃ CỔ PHIẾU ĐƯỢC ĐỀ CẬP NHIỀU NHẤT:\n")
    for i, (stock, count) in enumerate(stock_counter.most_common(20), 1):
        print(f"{i:2d}. {stock:6s} - {count:4d} lần")
else:
    print("\n⚠ Chưa có mã cổ phiếu nào được nhận diện")

## Các tính năng nâng cao

### Tìm kiếm tin tức theo mã cổ phiếu

In [ ]:
# Tìm kiếm tin tức theo mã cổ phiếu
def search_by_stock_code(df, stock_code):
    """
    Tìm tất cả tin tức liên quan đến một mã cổ phiếu
    """
    stock_code = stock_code.upper().strip()
    mask = df['stockCodes'].str.contains(stock_code, case=False, na=False)
    results = df[mask]
    
    print(f"🔍 Tìm thấy {len(results)} tin tức về mã {stock_code}\n")
    print(f"{'='*80}")
    
    for idx, row in results.iterrows():
        print(f"\n[{row['postID']}] {row['stockCodes']}")
        print(f"📅 {row['date']}")
        print(f"📌 {row['title']}")
        print(f"📝 {row['description']}")
        print(f"{'-'*80}")
    
    return results

# Ví dụ: Tìm tin về FPT
# search_by_stock_code(result_df, "FPT")+